In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage import io
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
    
import sklearn
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC


/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
def create_data_set(): 
    images_path = [ os.path.join("yalefaces", item)  for item in  os.listdir("yalefaces") ]
    image_data = []
    person_labels = []
    setting_labels = []
    
    for i,im_path in enumerate(images_path):
        im = io.imread(im_path,as_grey=True)
#         print(im_path)
#         io.imshow(im)
#         io.show()
        image_data.append(np.array(im, dtype='uint8'))
    
        person_label = int(os.path.split(im_path)[1].split(".")[0].replace("subject", "")) -1
        setting_label = os.path.split(im_path)[1].split(".")[1]

#         print(setting_label)
        person_labels.append(person_label)
        setting_labels.append(setting_label)
    return image_data, person_labels, setting_labels


In [0]:
x, y, dummy = create_data_set()

# reshaping
x = np.array(x)
y = np.array(y)


x_flat = np.reshape(x,(x.shape[0],77760))

x_norm = []

## normalizing the image for PCA
for i,image in enumerate(x):
#     print(i)
    image = StandardScaler().fit_transform(image)
    x_norm.append(np.array(image, dtype='uint8'))
    
x_norm = np.array(x_norm)
x_flat_norm = np.reshape(x_norm,(x_norm.shape[0],77760))
x_flat_norm = np.array(x_flat_norm)

print(x.shape)
print(y.shape)
print(x_flat_norm.shape)



C:\Users\tintin\Anaconda3\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')
C:\Users\tintin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(165, 243, 320)
(165,)
(165, 77760)


In [0]:
# svm predictor

## Cross validation to find num of components
num_of_components = [1,5,10,20,30,40,50]
index=0;
acc_array = []
count = 0

# finding the best params for SVC with RBF kernel
Cs= [1e3, 5e3, 1e4, 5e4, 1e5]
gammas = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]

# outer 5fold CV
kf_out = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)
# test train split
for train, test in kf_out.split(x_flat_norm, y):
        x_train, x_test = x_flat_norm[train], x_flat_norm[test]
        y_train, y_test = y[train], y[test]
#         print(x_train.shape)
#         print(y_train.shape)
#         print(y_train)
        # inner 5-fold CV
        fold =0
        max_acc = 0
        best_comp = 0
        best_C=0
        best_gamma=0
        for x in num_of_components:
#             print(x)
#             print(len(Cs))
#             print(len(gammas))
            for c_index in range(0,len(Cs)):
                for gamma_index in range(0,len(gammas)):
                    temp_acc_splits = []
                    kf_in = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 50)
                    for train_in, val in kf_in.split(x_train, y_train):
                        fold += 1
                        x_train_in, x_val = x_flat_norm[train_in], x_flat_norm[val]
                        y_train_in, y_val = y[train_in], y[val]   
            #             print(x_train_in.shape)
            #             print(x_val.shape) 
                        components = x
                        pca_in = PCA(n_components=components, whiten=True)
                        pca_in.fit(x_train_in)
                        x_train_in_pca = pca_in.transform(x_train_in)
                        x_val_pca = pca_in.transform(x_val)
                        clf = SVC(kernel='rbf',gamma=gammas[gamma_index], C = Cs[c_index])
#                         clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=d))
                        clf = clf.fit(x_train_in_pca, y_train_in)
                        y_pred = clf.predict(x_val_pca)
                        acc = metrics.accuracy_score(y_val, y_pred)
                        temp_acc_splits.append(acc)
                    accuracy_mean_splits = np.mean(np.array(temp_acc_splits))
                    if accuracy_mean_splits>max_acc:
                        max_acc=accuracy_mean_splits
                        best_comp = components
                        best_C= Cs[c_index]
                        best_gamma = gammas[gamma_index]
    #                 print(" Number of Components :",components , "best depth is ", best_depth ," and best Accuracy :",max_acc)

        # Now that we got the best model for each component from the inner CV use that on the test data
        print(best_comp)
        print(best_C)
        print(best_gamma)
        pca_out = PCA(n_components= best_comp, whiten=True)
        pca_out = pca_out.fit(x_train)
        x_train_pca = pca_out.transform(x_train)
#         print(x_train)
        x_test_pca = pca_out.transform(x_test)
        svc_out = SVC(kernel='rbf',gamma=best_gamma, C = best_C)
#         abc_out = AdaBoostClassifier(DecisionTreeClassifier(max_depth=best_depth))
#         print(y_train)
#         print(x_train_pca)
        svc_out = svc_out.fit(x_train_pca,y_train)
        y_pred_out = svc_out.predict(x_test_pca)
        acc_array.append(accuracy_score(y_test, y_pred_out))
        count +=1
        print("classification report in fold", count )
        print(metrics.classification_report(y_test, y_pred_out))
        
        
print("Average accuracy : ", np.mean(acc_array))

40
10000.0
0.0005
classification report in fold 1
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00         3
          2       0.75      1.00      0.86         3
          3       1.00      1.00      1.00         3
          4       1.00      1.00      1.00         3
          5       1.00      1.00      1.00         3
          6       1.00      1.00      1.00         3
          7       1.00      0.67      0.80         3
          8       1.00      1.00      1.00         3
          9       1.00      1.00      1.00         3
         10       1.00      1.00      1.00         3
         11       1.00      1.00      1.00         3
         12       1.00      1.00      1.00         3
         13       1.00      1.00      1.00         3
         14       1.00      1.00      1.00         3

avg / total       0.98      0.98      0.98        45

30
1000.0
0.0001
classification report in fold

C:\Users\tintin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


50
1000.0
0.0001
classification report in fold 5
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         2
          1       1.00      1.00      1.00         2
          2       1.00      1.00      1.00         2
          3       1.00      1.00      1.00         2
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         2
          6       1.00      1.00      1.00         2
          7       1.00      1.00      1.00         2
          8       1.00      1.00      1.00         2
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00         2
         11       1.00      1.00      1.00         2
         12       1.00      1.00      1.00         2
         13       1.00      1.00      1.00         2
         14       1.00      1.00      1.00         2

avg / total       1.00      1.00      1.00        30

Average accuracy :  0.9155555555555555
